In [1]:
param_x_api_key = '2BeBXg69E53RaoUFhA7hhf17jCg1NkwHoJz3FxmJPNS3bedbpHcb'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import math

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = '''{"getAutoCarePartApplications":
                {"brandCode":"''' + input_.loc[a, 'Brand_Id'] + '''",
                 "partNumber":"''' + input_.loc[a, 'Part_Number'] + '''",
                 "includeResults":true,
                 "includeMatchingBaseVehicles":true,
                 "perPage":100}}'''

                headers = UA.get_User_Agent_Requests()
                headers['X-Api-Key'] = param_x_api_key

                resp = requests.post('https://webservice.opticatonline.com/autocare/v1/services/Catalog.jsonEndpoint',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                vehicle_page = math.ceil(json_data['total'] / 100)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'JOIN_PNBI': input_.loc[a, 'JOIN_PNBI'],
                                         'Vehicle_Page': vehicle_page}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                     'Brand_Id': input_.loc[a, 'Brand_Id'],
                                     'JOIN_PNBI': input_.loc[a, 'JOIN_PNBI']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'JOIN_PNBI'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_PNBI'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle_page-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number', 'Brand_Id'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_page_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1285

[ok] - MF0227;GGNF
[尝试次数：1] - [剩余数量：1278] - [当前时间：20:23:55]

[ok] - MF0417;GGNF
[尝试次数：1] - [剩余数量：1277] - [当前时间：20:23:55]

[ok] - MF0415;GGNF
[尝试次数：1] - [剩余数量：1276] - [当前时间：20:23:55]

[ok] - MF0411;GGNF
[尝试次数：1] - [剩余数量：1275] - [当前时间：20:23:55]

[ok] - MF0421;GGNF
[尝试次数：1] - [剩余数量：1274] - [当前时间：20:23:56]

[ok] - MF0388;GGNF
[尝试次数：1] - [剩余数量：1273] - [当前时间：20:23:57]

[ok] - MF0930;GGNF
[尝试次数：1] - [剩余数量：1272] - [当前时间：20:23:57]

[ok] - MF0419;GGNF
[尝试次数：1] - [剩余数量：1271] - [当前时间：20:23:57]

[ok] - MF0917;GGNF
[尝试次数：1] - [剩余数量：1270] - [当前时间：20:23:57]

[ok] - MF20058;GGNF
[尝试次数：1] - [剩余数量：1269] - [当前时间：20:23:58]

[ok] - MF20046;GGNF
[尝试次数：1] - [剩余数量：1268] - [当前时间：20:23:58]

[ok] - MF0385;GGNF
[尝试次数：1] - [剩余数量：1267] - [当前时间：20:23:59]

[ok] - MF20059;GGNF
[尝试次数：1] - [剩余数量：1266] - [当前时间：20:23:59]

[ok] - MF0413;GGNF
[尝试次数：1] - [剩余数量：1265] - [当前时间：20:24:00]

[ok] - MF20065;GGNF
[尝试次数：1] - [剩余数量：1264] - [当前时间：20:24:02]

[ok] - MF20062;GGNF
[尝试次数：1] - [剩余数量：1263] - [当前时间：20:24:03]

[ok] - MF

[ok] - MF20056;GGNF
[尝试次数：1] - [剩余数量：1145] - [当前时间：20:25:02]

[ok] - MF20054;GGNF
[尝试次数：1] - [剩余数量：1144] - [当前时间：20:25:03]

[ok] - MF20057;GGNF
[尝试次数：1] - [剩余数量：1143] - [当前时间：20:25:04]

[ok] - MF20064;GGNF
[尝试次数：1] - [剩余数量：1142] - [当前时间：20:25:04]

[ok] - MF20053;GGNF
[尝试次数：1] - [剩余数量：1141] - [当前时间：20:25:04]

[ok] - MF20067;GGNF
[尝试次数：1] - [剩余数量：1140] - [当前时间：20:25:04]

[ok] - MF20070;GGNF
[尝试次数：1] - [剩余数量：1139] - [当前时间：20:25:05]

[ok] - MF20077;GGNF
[尝试次数：1] - [剩余数量：1138] - [当前时间：20:25:07]

[ok] - MF20080;GGNF
[尝试次数：1] - [剩余数量：1137] - [当前时间：20:25:08]

[ok] - MF20072;GGNF
[尝试次数：1] - [剩余数量：1136] - [当前时间：20:25:09]

[ok] - MF20090;GGNF
[尝试次数：1] - [剩余数量：1135] - [当前时间：20:25:09]

[ok] - MF20075;GGNF
[尝试次数：1] - [剩余数量：1134] - [当前时间：20:25:09]

[ok] - MF20102;GGNF
[尝试次数：1] - [剩余数量：1133] - [当前时间：20:25:10]

[ok] - MF20101;GGNF
[尝试次数：1] - [剩余数量：1132] - [当前时间：20:25:10]

[ok] - MF20103;GGNF
[尝试次数：1] - [剩余数量：1131] - [当前时间：20:25:12]

[ok] - MF20097;GGNF
[尝试次数：1] - [剩余数量：1130] - [当前时间：20:25:12]

[ok] - M

[ok] - MF21195;GGNF
[尝试次数：1] - [剩余数量：1011] - [当前时间：20:26:07]

[ok] - MF21196;GGNF
[尝试次数：1] - [剩余数量：1010] - [当前时间：20:26:07]

[ok] - MF21187;GGNF
[尝试次数：1] - [剩余数量：1009] - [当前时间：20:26:07]

[ok] - MF21198;GGNF
[尝试次数：1] - [剩余数量：1008] - [当前时间：20:26:07]

[ok] - MF21194;GGNF
[尝试次数：1] - [剩余数量：1007] - [当前时间：20:26:08]

[ok] - MF21200;GGNF
[尝试次数：1] - [剩余数量：1006] - [当前时间：20:26:08]

[ok] - MF21197;GGNF
[尝试次数：1] - [剩余数量：1005] - [当前时间：20:26:08]

[ok] - MF21204;GGNF
[尝试次数：1] - [剩余数量：1004] - [当前时间：20:26:09]

[ok] - MF21207;GGNF
[尝试次数：1] - [剩余数量：1003] - [当前时间：20:26:09]

[ok] - MF21202;GGNF
[尝试次数：1] - [剩余数量：1002] - [当前时间：20:26:09]

[ok] - MF21208;GGNF
[尝试次数：1] - [剩余数量：1001] - [当前时间：20:26:10]

[ok] - MF21206;GGNF
[尝试次数：1] - [剩余数量：1000] - [当前时间：20:26:10]

[ok] - MF21216;GGNF
[尝试次数：1] - [剩余数量：999] - [当前时间：20:26:11]

[ok] - MF21211;GGNF
[尝试次数：1] - [剩余数量：998] - [当前时间：20:26:11]

[ok] - MF21176;GGNF
[尝试次数：2] - [剩余数量：997] - [当前时间：20:26:11]

[ok] - MF21220;GGNF
[尝试次数：1] - [剩余数量：996] - [当前时间：20:26:11]

[ok] - MF212

[ok] - MF9650;GGNF
[尝试次数：1] - [剩余数量：875] - [当前时间：20:27:14]

[ok] - MF0862;GGNF
[尝试次数：1] - [剩余数量：874] - [当前时间：20:27:15]

[ok] - MF0864;GGNF
[尝试次数：1] - [剩余数量：873] - [当前时间：20:27:15]

[ok] - MF0887;GGNF
[尝试次数：1] - [剩余数量：872] - [当前时间：20:27:16]

[ok] - MF0863;GGNF
[尝试次数：1] - [剩余数量：871] - [当前时间：20:27:16]

[ok] - MF0892;GGNF
[尝试次数：1] - [剩余数量：870] - [当前时间：20:27:17]

[ok] - MF0872;GGNF
[尝试次数：1] - [剩余数量：869] - [当前时间：20:27:17]

[ok] - MF5450;GGNF
[尝试次数：1] - [剩余数量：868] - [当前时间：20:27:18]

[ok] - MF0876;GGNF
[尝试次数：1] - [剩余数量：867] - [当前时间：20:27:19]

[ok] - MF0895;GGNF
[尝试次数：1] - [剩余数量：866] - [当前时间：20:27:19]

[ok] - MF0897;GGNF
[尝试次数：1] - [剩余数量：865] - [当前时间：20:27:21]

[ok] - MF0912;GGNF
[尝试次数：1] - [剩余数量：864] - [当前时间：20:27:21]

[ok] - MF0916;GGNF
[尝试次数：1] - [剩余数量：863] - [当前时间：20:27:21]

[ok] - MF0914NS;GGNF
[尝试次数：1] - [剩余数量：862] - [当前时间：20:27:21]

[ok] - MF0928;GGNF
[尝试次数：1] - [剩余数量：861] - [当前时间：20:27:22]

[ok] - MF0926;GGNF
[尝试次数：1] - [剩余数量：860] - [当前时间：20:27:22]

[ok] - MF0901;GGNF
[尝试次数：1] - [剩余数量：85

[ok] - 221208;GGNG
[尝试次数：1] - [剩余数量：737] - [当前时间：20:29:24]

[ok] - 221234;GGNG
[尝试次数：1] - [剩余数量：736] - [当前时间：20:29:24]

[ok] - 221202;GGNG
[尝试次数：1] - [剩余数量：735] - [当前时间：20:29:24]

[ok] - 221220;GGNG
[尝试次数：1] - [剩余数量：734] - [当前时间：20:29:24]

[ok] - 221247;GGNG
[尝试次数：1] - [剩余数量：733] - [当前时间：20:29:25]

[ok] - 221249;GGNG
[尝试次数：1] - [剩余数量：732] - [当前时间：20:29:26]

[ok] - 221242;GGNG
[尝试次数：1] - [剩余数量：731] - [当前时间：20:29:26]

[ok] - 221246;GGNG
[尝试次数：1] - [剩余数量：730] - [当前时间：20:29:26]

[ok] - 221262;GGNG
[尝试次数：1] - [剩余数量：729] - [当前时间：20:29:27]

[ok] - 221252;GGNG
[尝试次数：1] - [剩余数量：728] - [当前时间：20:29:28]

[ok] - 221260;GGNG
[尝试次数：1] - [剩余数量：727] - [当前时间：20:29:28]

[ok] - 221271;GGNG
[尝试次数：1] - [剩余数量：726] - [当前时间：20:29:28]

[ok] - 221273;GGNG
[尝试次数：1] - [剩余数量：725] - [当前时间：20:29:29]

[ok] - 221266;GGNG
[尝试次数：1] - [剩余数量：724] - [当前时间：20:29:31]

[ok] - 221277;GGNG
[尝试次数：1] - [剩余数量：723] - [当前时间：20:29:31]

[ok] - 221278;GGNG
[尝试次数：1] - [剩余数量：722] - [当前时间：20:29:33]

[ok] - 221280;GGNG
[尝试次数：1] - [剩余数量：721]

[ok] - 59161;GHRL
[尝试次数：1] - [剩余数量：598] - [当前时间：20:30:46]

[ok] - 59162;GHRL
[尝试次数：1] - [剩余数量：597] - [当前时间：20:30:46]

[ok] - 59171;GHRL
[尝试次数：1] - [剩余数量：596] - [当前时间：20:30:47]

[ok] - 59173;GHRL
[尝试次数：1] - [剩余数量：595] - [当前时间：20:30:48]

[ok] - 59166;GHRL
[尝试次数：1] - [剩余数量：594] - [当前时间：20:30:48]

[ok] - 59174;GHRL
[尝试次数：1] - [剩余数量：593] - [当前时间：20:30:49]

[ok] - 59175;GHRL
[尝试次数：1] - [剩余数量：592] - [当前时间：20:30:50]

[ok] - 59178;GHRL
[尝试次数：1] - [剩余数量：591] - [当前时间：20:30:51]

[ok] - 59130;GHRL
[尝试次数：1] - [剩余数量：590] - [当前时间：20:30:51]

[ok] - 59159;GHRL
[尝试次数：1] - [剩余数量：589] - [当前时间：20:30:53]

[ok] - 59183;GHRL
[尝试次数：1] - [剩余数量：588] - [当前时间：20:30:53]

[ok] - 59191;GHRL
[尝试次数：1] - [剩余数量：587] - [当前时间：20:30:53]

[ok] - 59194;GHRL
[尝试次数：1] - [剩余数量：586] - [当前时间：20:30:54]

[ok] - 59176;GHRL
[尝试次数：1] - [剩余数量：585] - [当前时间：20:30:55]

[ok] - 59198;GHRL
[尝试次数：1] - [剩余数量：584] - [当前时间：20:30:55]

[ok] - 59197;GHRL
[尝试次数：1] - [剩余数量：583] - [当前时间：20:30:56]

[ok] - 59195;GHRL
[尝试次数：1] - [剩余数量：582] - [当前时间：20:30:56

[ok] - 28775;GGNG
[尝试次数：1] - [剩余数量：459] - [当前时间：20:32:34]

[ok] - 29011;GGNG
[尝试次数：1] - [剩余数量：458] - [当前时间：20:32:35]

[ok] - 29025;GGNG
[尝试次数：1] - [剩余数量：457] - [当前时间：20:32:35]

[ok] - 29021;GGNG
[尝试次数：1] - [剩余数量：456] - [当前时间：20:32:35]

[ok] - 29028;GGNG
[尝试次数：1] - [剩余数量：455] - [当前时间：20:32:36]

[ok] - 29022;GGNG
[尝试次数：1] - [剩余数量：454] - [当前时间：20:32:36]

[ok] - 29052;GGNG
[尝试次数：1] - [剩余数量：453] - [当前时间：20:32:36]

[ok] - 29080;GGNG
[尝试次数：1] - [剩余数量：452] - [当前时间：20:32:37]

[ok] - 29082;GGNG
[尝试次数：1] - [剩余数量：451] - [当前时间：20:32:37]

[ok] - 29091;GGNG
[尝试次数：1] - [剩余数量：450] - [当前时间：20:32:37]

[ok] - 29085;GGNG
[尝试次数：1] - [剩余数量：449] - [当前时间：20:32:38]

[ok] - 29129;GGNG
[尝试次数：1] - [剩余数量：448] - [当前时间：20:32:39]

[ok] - 29130;GGNG
[尝试次数：1] - [剩余数量：447] - [当前时间：20:32:40]

[ok] - 29029;GGNG
[尝试次数：1] - [剩余数量：446] - [当前时间：20:32:40]

[ok] - 29132;GGNG
[尝试次数：1] - [剩余数量：445] - [当前时间：20:32:41]

[ok] - 29139;GGNG
[尝试次数：1] - [剩余数量：444] - [当前时间：20:32:41]

[ok] - 57870;GHRL
[尝试次数：1] - [剩余数量：443] - [当前时间：20:32:42

[ok] - 221203;GGNG
[尝试次数：1] - [剩余数量：319] - [当前时间：20:33:30]

[ok] - 221224;GGNG
[尝试次数：1] - [剩余数量：318] - [当前时间：20:33:30]

[ok] - 20427;GGNG
[尝试次数：1] - [剩余数量：317] - [当前时间：20:33:30]

[ok] - 221163;GGNG
[尝试次数：1] - [剩余数量：316] - [当前时间：20:33:31]

[ok] - 22884M;GGNG
[尝试次数：1] - [剩余数量：315] - [当前时间：20:33:31]

[ok] - 221209;GGNG
[尝试次数：1] - [剩余数量：314] - [当前时间：20:33:32]

[ok] - 23986M;GGNG
[尝试次数：1] - [剩余数量：313] - [当前时间：20:33:33]

[ok] - 221258;GGNG
[尝试次数：1] - [剩余数量：312] - [当前时间：20:33:34]

[ok] - 221359;GGNG
[尝试次数：1] - [剩余数量：311] - [当前时间：20:33:35]

[ok] - 26313M;GGNG
[尝试次数：1] - [剩余数量：310] - [当前时间：20:33:35]

[ok] - 27795;GGNG
[尝试次数：1] - [剩余数量：309] - [当前时间：20:33:35]

[ok] - 27854;GGNG
[尝试次数：1] - [剩余数量：308] - [当前时间：20:33:36]

[ok] - 27850;GGNG
[尝试次数：1] - [剩余数量：307] - [当前时间：20:33:36]

[ok] - 27856;GGNG
[尝试次数：1] - [剩余数量：306] - [当前时间：20:33:38]

[ok] - 27799;GGNG
[尝试次数：1] - [剩余数量：305] - [当前时间：20:33:38]

[ok] - 27857;GGNG
[尝试次数：1] - [剩余数量：304] - [当前时间：20:33:38]

[ok] - 27860;GGNG
[尝试次数：1] - [剩余数量：303] - [当前时间

[ok] - 621352A;GHRL
[尝试次数：1] - [剩余数量：182] - [当前时间：20:35:18]

[ok] - 622005F;GHRL
[尝试次数：1] - [剩余数量：181] - [当前时间：20:35:18]

[ok] - 622008F;GHRL
[尝试次数：1] - [剩余数量：180] - [当前时间：20:35:18]

[ok] - 650021F;GHRL
[尝试次数：1] - [剩余数量：179] - [当前时间：20:35:19]

[ok] - 650003A;GHRL
[尝试次数：1] - [剩余数量：178] - [当前时间：20:35:20]

[ok] - 650023F;GHRL
[尝试次数：1] - [剩余数量：177] - [当前时间：20:35:21]

[ok] - 650022A;GHRL
[尝试次数：1] - [剩余数量：176] - [当前时间：20:35:21]

[ok] - 622002A;GHRL
[尝试次数：1] - [剩余数量：175] - [当前时间：20:35:22]

[ok] - 650017F;GHRL
[尝试次数：1] - [剩余数量：174] - [当前时间：20:35:22]

[ok] - 20425M;GGNG
[尝试次数：1] - [剩余数量：173] - [当前时间：20:35:23]

[ok] - 650024F;GHRL
[尝试次数：1] - [剩余数量：172] - [当前时间：20:35:24]

[ok] - 213350M;GGNG
[尝试次数：1] - [剩余数量：171] - [当前时间：20:35:25]

[ok] - 212543M;GGNG
[尝试次数：1] - [剩余数量：170] - [当前时间：20:35:26]

[ok] - 214383M;GGNG
[尝试次数：1] - [剩余数量：169] - [当前时间：20:35:27]

[ok] - 220020M;GGNG
[尝试次数：1] - [剩余数量：168] - [当前时间：20:35:28]

[ok] - 220035M;GGNG
[尝试次数：1] - [剩余数量：167] - [当前时间：20:35:29]

[ok] - 219352M;GGNG
[尝试次数

[ok] - 221028M;GGNG
[尝试次数：1] - [剩余数量：44] - [当前时间：20:37:59]

[ok] - 221033M;GGNG
[尝试次数：1] - [剩余数量：43] - [当前时间：20:38:00]

[ok] - 221031M;GGNG
[尝试次数：1] - [剩余数量：42] - [当前时间：20:38:00]

[ok] - 221005M;GGNG
[尝试次数：1] - [剩余数量：41] - [当前时间：20:38:00]

[ok] - 221040M;GGNG
[尝试次数：1] - [剩余数量：40] - [当前时间：20:38:01]

[ok] - 221038M;GGNG
[尝试次数：1] - [剩余数量：39] - [当前时间：20:38:02]

[ok] - 221072M;GGNG
[尝试次数：1] - [剩余数量：38] - [当前时间：20:38:03]

[ok] - 221070M;GGNG
[尝试次数：1] - [剩余数量：37] - [当前时间：20:38:04]

[ok] - 221085M;GGNG
[尝试次数：1] - [剩余数量：36] - [当前时间：20:38:05]

[ok] - 221044M;GGNG
[尝试次数：1] - [剩余数量：35] - [当前时间：20:38:05]

[ok] - 221083M;GGNG
[尝试次数：1] - [剩余数量：34] - [当前时间：20:38:05]

[ok] - 23984M;GGNG
[尝试次数：1] - [剩余数量：33] - [当前时间：20:38:06]

[ok] - 23985M;GGNG
[尝试次数：1] - [剩余数量：32] - [当前时间：20:38:06]

[ok] - 221129M;GGNG
[尝试次数：1] - [剩余数量：31] - [当前时间：20:38:07]

[ok] - 25288M;GGNG
[尝试次数：1] - [剩余数量：30] - [当前时间：20:38:07]

[ok] - 25450M;GGNG
[尝试次数：1] - [剩余数量：29] - [当前时间：20:38:07]

[ok] - 25519M;GGNG
[尝试次数：1] - [剩余数量：28] - [当